In [1]:
import logging
import os
import tensorflow as tf
import gym
from baselines import logger
from baselines.common import set_global_seeds
from baselines import bench
from baselines.acktr.acktr_cont import learn
from baselines.acktr.policies import GaussianMlpPolicy
from baselines.acktr.value_functions import NeuralNetValueFunction

In [2]:
import gym
import gym_minecraft
from MinecraftGym import MinecraftWrapper

In [5]:
from keras import backend as K
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Activation,GRU,Input,LSTM,Conv2D,Flatten
from keras.optimizers import RMSprop,Adam
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [3]:
pre_env = gym.make("MinecraftCliffWalking1-v0")
pre_env.init(videoResolution=[400,400],allowContinuousMovement=["move", "turn", "strafe"])
env = MinecraftWrapper(pre_env,scale=1/10)

In [6]:
## create an image based value function using CNN

model = Sequential()
model.add(Conv2D(32,(8,8),input_shape=env.observation_space.shape,activation='relu'))
model.add(Conv2D(64,(4,4),activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Flatten())
model.add(Dense(60,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(output_dim=env.action_space.n,activation='softmax'))

/home/ubuntu/miniconda2/envs/casper/lib/python3.5/site-packages/ipykernel/__main__.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=6, activation="softmax")`


In [8]:

def train(num_timesteps):
    #env=gym.make(env_id)
    #env = bench.Monitor(env, logger.get_dir() and os.path.join(logger.get_dir(), str(rank)))
    #set_global_seeds(seed)
    #env.seed(seed)
    gym.logger.setLevel(logging.WARN)

    with tf.Session(config=tf.ConfigProto()):
        ob_dim = env.observation_space.shape[0]
        ac_dim = env.action_space.n
        with tf.variable_scope("vf"):
            vf = model.output
        with tf.variable_scope("pi"):
            policy = GaussianMlpPolicy(ob_dim, ac_dim)

        learn(env, policy=policy, vf=vf,
            gamma=0.99, lam=0.97, timesteps_per_batch=2500,
            desired_kl=0.002,
            num_timesteps=num_timesteps, animate=False)

    env.close()

In [9]:
train(int(1e6))

Use async eigen decomp
updating 14 eigenvalue/vectors
projecting 4 gradient matrices


AttributeError: 'Tensor' object has no attribute 'q_runner'